<a href="https://colab.research.google.com/github/hiraso/lab_task/blob/master/%E7%AC%AC5%E5%9B%9E%E8%AA%B2%E9%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第5回課題
第5回課題では，以下の項目を満たすように分析してください．

明確な解答は用意していませんが，少なくとも間違っていないと言える推論をするようにしてください．

- データの取得
- データの確認
 - 特徴量の確認
 - ラベルの確認
- 仮説を立てる
- データの分析
- モデルの訓練
- 予測
- 精度の確認
- 仮説の確認

# データの取得
wine data 以外を利用してデータ分析してください．

なるべく，ラベルと属性から推論が立てやすいデータを選択するとやりやすいです．

In [0]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston

In [0]:
data = load_boston()

# データの確認
データ分析以前にわかることを確認．特徴量の属性やドメイン知識など
- 特徴量の確認
- ラベルの確認

In [3]:
data.keys()

dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename'])

In [4]:
data["feature_names"]

array(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD',
       'TAX', 'PTRATIO', 'B', 'LSTAT'], dtype='<U7')

In [0]:
#CRIM 犯罪率
#ZN　区画率
#INDUS　非小売業の割合
#CHAS 川に面していたら１　面してなかったら０
#NOX　窒素酸化物濃度
#RM　部屋数の平均
#AGE　1940年に建てられた家の割合
#DIS　5つのボストン雇用センターまでの距離の加重
#RAD　環状高速道路の数
#TAX　固定資産税率
#PTRATIO　生徒と教師の割合
#B　黒人の割合
#LSTAT　人口減少の度合い

# 仮説を立てる
データについて，導きたい仮説と結論を整理してください．

固定資産税が高いと、住宅価格が高くなると予想した。
また、住宅価格が高いものは、固定資産税も高いと予想した。
また、固定資産税率が高い条件に犯罪率、平均の部屋の数が関わっていると予想した。

# データの分析

In [0]:
boston_df = pd.DataFrame(data["data"], columns=data["feature_names"])
boston_df = pd.concat([boston_df, pd.DataFrame(data["target"], columns=["target"])], axis=1)

In [7]:
boston_df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [8]:
boston_df.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


In [9]:
boston_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    float64
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    float64
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  target   506 non-null    float64
dtypes: float64(14)
memory usage: 55.5 KB


In [0]:
#欠損値がないことがわかった

In [0]:
variable = data["feature_names"]

In [12]:
[print(str(i) + str(boston_df[i].unique().shape)) for i in variable]

CRIM(504,)
ZN(26,)
INDUS(76,)
CHAS(2,)
NOX(81,)
RM(446,)
AGE(356,)
DIS(412,)
RAD(9,)
TAX(66,)
PTRATIO(46,)
B(357,)
LSTAT(455,)


[None, None, None, None, None, None, None, None, None, None, None, None, None]

In [0]:
#CHASは２値のcategorical変数である
#RADから最大の高速道路の個数は９個
#また、TAX（固定資産税）が６６種類あることから何軒か同じ条件が揃っている
#固定資産税について調べてみたところ、住むのにいい条件が揃っていると高くなるものであったため、犯罪率、平均の部屋の数などは快適、便利な条件と受け取れるため重要なものと考える。

In [0]:
#そのため、犯罪率と平均の部屋の数、固定資産税率の値を２乗することで重要性を挙げてみる。
#ただ、正当性のない憶測なので２乗したデータとしてないデータの２種類を使い比較する。

In [0]:
boston_df = pd.concat([boston_df, pd.get_dummies(boston_df["CHAS"], prefix="CHAS")], axis=1)

In [0]:
variable =['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD','TAX', 'PTRATIO', 'B', 'LSTAT',"CHAS_0.0","CHAS_1.0"]

In [17]:
boston_df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target,CHAS_0.0,CHAS_1.0
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0,1,0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6,1,0
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7,1,0
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4,1,0
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2,1,0


In [0]:
boston2_df = boston_df
variable_CRT = ["CRIM","RM","TAX"]
boston2_df["CRIM"] = boston2_df["CRIM"] * boston2_df["CRIM"]
boston2_df["RM"] = boston2_df["RM"] * boston2_df["RM"]
boston2_df["TAX"] = boston2_df["TAX"] * boston2_df["TAX"]

In [19]:
boston2_df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target,CHAS_0.0,CHAS_1.0
0,0.000040,18.0,2.31,0.0,0.538,43.230625,65.2,4.0900,1.0,87616.0,15.3,396.90,4.98,24.0,1,0
1,0.000746,0.0,7.07,0.0,0.469,41.229241,78.9,4.9671,2.0,58564.0,17.8,396.90,9.14,21.6,1,0
2,0.000745,0.0,7.07,0.0,0.469,51.624225,61.1,4.9671,2.0,58564.0,17.8,392.83,4.03,34.7,1,0
3,0.001048,0.0,2.18,0.0,0.458,48.972004,45.8,6.0622,3.0,49284.0,18.7,394.63,2.94,33.4,1,0
4,0.004768,0.0,2.18,0.0,0.458,51.079609,54.2,6.0622,3.0,49284.0,18.7,396.90,5.33,36.2,1,0


In [0]:
df_corr = boston_df.corr()
df_corr2 = boston2_df.corr()

In [21]:
df_corr

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target,CHAS_0.0,CHAS_1.0
CRIM,1.000000,-0.082634,0.172073,-0.041163,0.174481,-0.080254,0.158977,-0.174295,0.279830,0.266061,0.135151,-0.191856,0.209275,-0.218837,0.041163,-0.041163
ZN,-0.082634,1.000000,-0.533828,-0.042697,-0.516604,0.307863,-0.569537,0.664408,-0.311948,-0.314988,-0.391679,0.175520,-0.412995,0.360445,0.042697,-0.042697
INDUS,0.172073,-0.533828,1.000000,0.062938,0.763651,-0.382654,0.644779,-0.708027,0.595129,0.716358,0.383248,-0.356977,0.603800,-0.483725,-0.062938,0.062938
CHAS,-0.041163,-0.042697,0.062938,1.000000,0.091203,0.098034,0.086518,-0.099176,-0.007368,-0.033518,-0.121515,0.048788,-0.053929,0.175260,-1.000000,1.000000
NOX,0.174481,-0.516604,0.763651,0.091203,1.000000,-0.283930,0.731470,-0.769230,0.611441,0.659892,0.188933,-0.380051,0.590879,-0.427321,-0.091203,0.091203
RM,-0.080254,0.307863,-0.382654,0.098034,-0.283930,1.000000,-0.222991,0.183715,-0.196734,-0.269352,-0.365443,0.125611,-0.600346,0.718169,-0.098034,0.098034
AGE,0.158977,-0.569537,0.644779,0.086518,0.731470,-0.222991,1.000000,-0.747881,0.456022,0.503659,0.261515,-0.273534,0.602339,-0.376955,-0.086518,0.086518
DIS,-0.174295,0.664408,-0.708027,-0.099176,-0.769230,0.183715,-0.747881,1.000000,-0.494588,-0.537830,-0.232471,0.291512,-0.496996,0.249929,0.099176,-0.099176
RAD,0.279830,-0.311948,0.595129,-0.007368,0.611441,-0.196734,0.456022,-0.494588,1.000000,0.932824,0.464741,-0.444413,0.488676,-0.381626,0.007368,-0.007368
TAX,0.266061,-0.314988,0.716358,-0.033518,0.659892,-0.269352,0.503659,-0.537830,0.932824,1.000000,0.477511,-0.444892,0.544425,-0.457047,0.033518,-0.033518


In [22]:
df_corr2

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target,CHAS_0.0,CHAS_1.0
CRIM,1.000000,-0.082634,0.172073,-0.041163,0.174481,-0.080254,0.158977,-0.174295,0.279830,0.266061,0.135151,-0.191856,0.209275,-0.218837,0.041163,-0.041163
ZN,-0.082634,1.000000,-0.533828,-0.042697,-0.516604,0.307863,-0.569537,0.664408,-0.311948,-0.314988,-0.391679,0.175520,-0.412995,0.360445,0.042697,-0.042697
INDUS,0.172073,-0.533828,1.000000,0.062938,0.763651,-0.382654,0.644779,-0.708027,0.595129,0.716358,0.383248,-0.356977,0.603800,-0.483725,-0.062938,0.062938
CHAS,-0.041163,-0.042697,0.062938,1.000000,0.091203,0.098034,0.086518,-0.099176,-0.007368,-0.033518,-0.121515,0.048788,-0.053929,0.175260,-1.000000,1.000000
NOX,0.174481,-0.516604,0.763651,0.091203,1.000000,-0.283930,0.731470,-0.769230,0.611441,0.659892,0.188933,-0.380051,0.590879,-0.427321,-0.091203,0.091203
RM,-0.080254,0.307863,-0.382654,0.098034,-0.283930,1.000000,-0.222991,0.183715,-0.196734,-0.269352,-0.365443,0.125611,-0.600346,0.718169,-0.098034,0.098034
AGE,0.158977,-0.569537,0.644779,0.086518,0.731470,-0.222991,1.000000,-0.747881,0.456022,0.503659,0.261515,-0.273534,0.602339,-0.376955,-0.086518,0.086518
DIS,-0.174295,0.664408,-0.708027,-0.099176,-0.769230,0.183715,-0.747881,1.000000,-0.494588,-0.537830,-0.232471,0.291512,-0.496996,0.249929,0.099176,-0.099176
RAD,0.279830,-0.311948,0.595129,-0.007368,0.611441,-0.196734,0.456022,-0.494588,1.000000,0.932824,0.464741,-0.444413,0.488676,-0.381626,0.007368,-0.007368
TAX,0.266061,-0.314988,0.716358,-0.033518,0.659892,-0.269352,0.503659,-0.537830,0.932824,1.000000,0.477511,-0.444892,0.544425,-0.457047,0.033518,-0.033518


# モデルの訓練
データは分割するようにしてください．交叉検証についてはまだ説明したいので自由にしてください．

また，モデルの訓練およびデータ分割時に乱数の指定がないなど再現性がなさそうなコードは認めないことにします．

モデルは自由ですが，sklearn を想定しています．自作ライブラリも可ですが，自作の場合，該当箇所は適当に読みます．

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor  as RFR

In [24]:
#データの１部を２乗したデータの訓練
X2_train, X2_test, y2_train, y2_test = train_test_split(boston２_df[variable], boston２_df["target"], test_size=0.3, random_state=43)
clf2 = RFR(n_jobs=-1, random_state=43)
clf2.fit(X2_train, y2_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=-1, oob_score=False,
                      random_state=43, verbose=0, warm_start=False)

In [0]:
#そのままのデータの訓練
boston_df = pd.DataFrame(data["data"], columns=data["feature_names"])
boston_df = pd.concat([boston_df, pd.DataFrame(data["target"], columns=["target"])], axis=1)
boston_df = pd.concat([boston_df, pd.get_dummies(boston_df["CHAS"], prefix="CHAS")], axis=1)
X_train, X_test, y_train, y_test = train_test_split(boston_df[variable], boston_df["target"], test_size=0.3, random_state=43)

In [26]:
clf = RFR(n_jobs=-1, random_state=43)
clf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=-1, oob_score=False,
                      random_state=43, verbose=0, warm_start=False)

# 予測

In [0]:
#データの１部を変更した方
pred2 = clf2.predict(X2_test)
#データそのまま
pred = clf.predict(X_test)

In [28]:
pred

array([24.984, 15.83 , 20.455, 30.657, 18.384, 17.012, 18.194, 23.736,
       21.075, 17.204, 21.883, 26.195, 17.705, 20.225, 18.718,  9.42 ,
       26.83 , 33.33 , 13.149, 46.533, 20.235,  8.553, 20.286, 34.196,
        8.616, 26.93 , 15.86 , 19.923, 20.385, 14.172, 20.718, 20.341,
       15.122, 11.618, 20.181, 19.57 , 14.898, 20.108, 26.377, 24.795,
       19.367,  9.763, 22.005, 20.661, 46.086, 19.647, 21.615, 18.643,
       43.167, 14.564, 26.94 , 45.365, 43.954, 21.386, 14.703, 49.055,
       42.09 , 23.375, 27.696, 26.568, 20.167, 15.234, 22.126, 23.267,
       29.928, 14.36 , 20.642, 34.401,  9.543, 22.982, 15.785,  8.745,
        9.227, 12.11 , 14.977, 23.526, 20.575, 22.799, 24.057, 19.407,
       12.277, 20.189, 14.959,  9.77 , 18.137, 26.675, 12.167, 33.891,
       11.943, 13.98 , 15.492, 20.933, 21.123, 22.41 , 20.493, 17.915,
       33.227, 32.431, 32.322, 44.08 , 15.79 , 20.52 ,  9.111, 27.377,
       20.564, 15.153, 14.608, 19.623, 19.89 , 33.953, 12.878, 26.879,
      

In [29]:
pred2

array([24.988, 15.828, 20.465, 30.657, 18.394, 17.012, 18.344, 23.732,
       21.075, 17.204, 21.878, 26.195, 17.705, 20.225, 18.719,  9.42 ,
       26.83 , 33.33 , 13.149, 46.533, 20.225,  8.553, 20.286, 34.196,
        8.616, 26.72 , 15.886, 19.923, 20.385, 14.172, 20.731, 20.33 ,
       15.122, 11.618, 20.181, 19.57 , 14.898, 20.108, 26.403, 24.795,
       19.352,  9.874, 22.005, 20.363, 46.091, 19.647, 21.615, 18.643,
       43.173, 14.564, 26.94 , 45.433, 43.954, 21.374, 14.703, 49.069,
       41.84 , 23.375, 27.696, 26.568, 20.167, 15.259, 22.116, 23.267,
       29.928, 14.36 , 20.643, 34.399,  9.543, 22.982, 15.785,  8.746,
        9.211, 12.11 , 14.977, 23.533, 20.571, 22.799, 24.048, 19.407,
       12.277, 20.189, 14.959,  9.779, 18.154, 26.675, 12.167, 33.891,
       11.978, 13.98 , 15.51 , 20.933, 21.1  , 22.42 , 20.489, 17.915,
       33.227, 32.301, 32.322, 44.08 , 15.791, 20.52 ,  9.114, 27.377,
       20.564, 15.206, 14.616, 19.641, 19.89 , 33.953, 12.871, 26.879,
      

# 精度の確認

In [30]:
#データを変更した方
clf2.score(X2_test,y2_test)

0.8863044184800977

In [31]:
#そのままのデータ
clf.score(X_test, y_test)

0.8862967228318026

# 仮説の確認
最初に立てた仮説を確認してください．

予測傾向など精度ベースでも仮説を確認することはできますが，決定木や可視化を用いる方が楽かもしれません．

In [0]:
#今回は、TAX,RM,CRIMを重要視するようなモデルを作ったが、そのままのデータから少し精度が上がっただけでほぼ変わらないことがわかった。